In [1]:
import cv2

# Load the pre-trained Haar Cascade classifiers for pedestrian and full-body detection
pedestrian_cascade = cv2.CascadeClassifier('fullbody.xml')
fullbody_cascade = cv2.CascadeClassifier('upperbody.xml')

# Load video file or capture from webcam
video_source = "people.mp4"  # Set the video source path or use 0 for webcam
cap = cv2.VideoCapture(video_source)

# Initialize variables
people_count = 0
group_count = 0
group_threshold = 20  # Threshold for considering a group
message = ""

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to grayscale for pedestrian and full-body detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect pedestrians in the frame
    pedestrians = pedestrian_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Detect full-body in the frame
    fullbodies = fullbody_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Combine the results from both detectors
    all_people = list(pedestrians) + list(fullbodies)

    # Initialize variables for each frame
    frame_group_count = 0

    # Draw rectangles around detected people
    for (x, y, w, h) in all_people:
        if w * h > 1000:  # Filter out small detections
            if frame_group_count == 0:
                group_count += 1
            frame_group_count += 1
            color = (0, 0, 255) if frame_group_count > group_threshold else (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    # Update the people count and message
    people_count = len(all_people)
    if frame_group_count > group_threshold:
        message = "Crowd Formed"
    elif frame_group_count < group_threshold:
        message = "The Area is in normal state"
    else:
        message = ""

    # Display the count and message on the frame
    cv2.putText(frame, f"People Count: {people_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, message, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the frame with detected people
    cv2.imshow("People Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
